<a href="https://colab.research.google.com/github/Untick/Marketplaces/blob/Kosmylev-Vladislav-branch/3_svod_DataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка

## Загрузка исходников

####**Загрузка библиотек:**

https://drive.google.com/file/d/1-_LKbDsiq0dDbks7_JnBJVV7v2iWjSkd/view?usp=sharing

https://drive.google.com/file/d/1CzIZl5hxOpNsEiWkcJgjXRsAVJz4RbsX/view?usp=sharing  target_tab.zip

In [3]:
import numpy as np                        # Импорт библиотеки numpy
import pandas as pd                       # для анализа табличных массивов данных
from pandas.core.groupby import NamedAgg
import gdown, os, re, random                  # Стандартные библиотеки

In [4]:
!gdown --id '1-_LKbDsiq0dDbks7_JnBJVV7v2iWjSkd'
#!gdown --id '1CzIZl5hxOpNsEiWkcJgjXRsAVJz4RbsX'
!rm -r sample_data
#!rm base_for_analiz.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-_LKbDsiq0dDbks7_JnBJVV7v2iWjSkd
To: /content/base_for_analiz.zip
100% 88.5M/88.5M [00:01<00:00, 63.5MB/s]
rm: cannot remove 'sample_data': No such file or directory


In [5]:
!unzip -o /content/base_for_analiz.zip
# !unzip -o /content/target_tab.zip
# !rm target_tab.zip
!rm base_for_analiz.zip

Archive:  /content/base_for_analiz.zip
  inflating: base_for_analiz/MPSTATS/Категория/MPSTATS Категория по дням/OZON - Автотовары_Автозапчасти_Автосвет - По дням - 01.03.2023-31.05.2023. (06.06.2023).csv  
  inflating: base_for_analiz/MPSTATS/Категория/MPSTATS Категория по дням/OZON - Автотовары_Автозапчасти_Автосвет - По дням - 01.06.2022-31.08.2022. (06.06.2023).csv  
  inflating: base_for_analiz/MPSTATS/Категория/MPSTATS Категория по дням/OZON - Автотовары_Автозапчасти_Автосвет - По дням - 01.09.2022-30.11.2022. (06.06.2023).csv  
  inflating: base_for_analiz/MPSTATS/Категория/MPSTATS Категория по дням/OZON - Автотовары_Автозапчасти_Автосвет - По дням - 01.12.2022-28.02.2023. (06.06.2023).csv  
 extracting: base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)/OZON - Автотовары_Автозапчасти_Автосвет - Товары - 01032023-31052023 06062023.csv.zip  
 extracting: base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)/OZON - Автотовары_Автозапчаст

In [6]:
def travel_dir(folder_path, subdir=True):
    file_dict = {}
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            file_name = os.path.splitext(file)[0]
            file_dict[file_name] = file_path
        if not subdir:
          return file_dict
    return file_dict


In [7]:
import re

def clean_column_names(df):
    cleaned_columns = []

    for column in df.columns:
        cleaned_column = re.sub(r"[^\w\s%]", "", column)  # Оставляем только буквы, цифры и пробелы
        cleaned_column = cleaned_column.replace(" ", "_")  # Заменяем пробелы на "_"
        if cleaned_column[-1] == '_':
            cleaned_column = cleaned_column[:-1]
        cleaned_columns.append(cleaned_column)
    return cleaned_columns
    # df.columns = cleaned_columns
    # return df

def normal_col(df):
  cols = clean_column_names(df)
  print(cols)
  df.set_axis(cols, axis=1, inplace=True)

In [8]:
def float2int(df):
  col_float = list(df.select_dtypes(include=['float64']).columns)
  df.fillna(0, inplace=True)
  for col in col_float:
    df[col]=df[col].round(0).astype('int64')

In [9]:
# Нормализуем заголовки столбцов
remove_list = ["(при наличии)",                 # список кусков текста для удаления из названий столбцов
               "(кроме Твери)",
               "руб.",
               "(перечеркнутая цена)",
               ",",
               "%",
               "с учетом скидки",
               "на рыночную цену"]
def clean_header(header):

    cleaned_header = header
    cleaned_header = cleaned_header.replace('-', "_")
    cleaned_header = cleaned_header.replace('" ', '"')
    cleaned_header = cleaned_header.replace(' "', '"')

    for item in remove_list:
        cleaned_header = cleaned_header.replace(item, "")

    cleaned_header = cleaned_header.replace('  ', '')
    cleaned_header = cleaned_header.replace('_"', '"')
    cleaned_header = cleaned_header.strip()  # удаление ведущих и концевых пробелов
    cleaned_header = cleaned_header.replace(" ", "_")  # замена оставшихся пробелов символом "_"
    cleaned_header = cleaned_header.replace('_"', '"')

    list_heads = cleaned_header.replace(';', ',')
    list_heads = list_heads.replace('"', '')
    list_heads = list_heads.replace('_"', '"')
    # print(header, " --->  ", list_heads)

    return cleaned_header

def clean_csv(file_path):
    with open(file_path, 'r', newline='') as file:
        lines = file.readlines()
        cleaned_headers = clean_header(lines[0]);  # применяем функцию clean_header к каждому заголовку
        lines[0] = cleaned_headers + '\n'
        print(cleaned_headers)

    with open(file_path, 'w', newline='') as file:
        file.writelines(lines)

    return cleaned_headers


#### Формируем ДатаФреймы

##### Пути по сегментам

In [10]:
fdir = '/content/base_for_analiz/Ozon/Товары по категориям'
cat_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/Ozon/Performance/Трафареты'
traf_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/Ozon/Performance/ПВП'
pvp_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/Ozon'
ozon_paths = travel_dir(fdir, False)

fdir = '/content/base_for_analiz/MPSTATS/Категория/MPSTATS Категория по дням'
mpcat_day_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)'
mpcat_sel_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/MPSTATS/Магазин/MPSTATS 01062022-31052023 Выручка по дням'
mpmag_gros_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/MPSTATS/Магазин/MPSTATS 01062022-31052023 Товары'
mpmag_tov_paths = travel_dir(fdir)


##### Загрузка исходников в ДатаФреймы

In [11]:
all_df = {}

#### Ozon -> DF

загружаем индивидуально из-за особенностей данных

###### База

In [12]:
os.getcwd()
os.chdir('/content/base_for_analiz/Ozon/')

In [13]:
os.getcwd()

'/content/base_for_analiz/Ozon'

In [14]:
columns = ['Артикул','Ozon_Product_ID','FBO_OZON_SKU_ID','FBS_OZON_SKU_ID','Контент_рейтинг','Текущая_цена']
xls = 'База.csv'
list_heads = clean_csv(xls)
df = pd.read_csv(xls, sep=';', encoding='utf-8', usecols=columns) #


﻿"Артикул";"Ozon_Product_ID";"FBO_OZON_SKU_ID";"FBS_OZON_SKU_ID";"Barcode";"Наименование_товара";"Контент_рейтинг";"Бренд";"Статус_товара";"Видимость_FBO";"Причины_скрытия_FBO";"Видимость_FBS";"Причины_скрытия_FBS";"Дата_создания";"Коммерческая_категория";"Объем_товара_л";"Объемный_вес_кг";"Доступно_на_складе_Ozon_шт";"Вывезти_и_нанести_КИЗшт";"Зарезервировано_шт";"Доступно_на_моих_складах_шт";"Зарезервировано_на_моих_складах_шт";"Текущая_цена";"Цена_до_скидки";"Цена_Premium";"Рыночная_цена";"Актуальная_ссылка";"Размер_НДС"


In [15]:
df = df[df['Контент_рейтинг'] != "'Не рассчитываем контент-рейтинг для уценки FBS"].copy()

In [16]:
df['Контент_рейтинг'] = df['Контент_рейтинг'].replace('Рассчитываем',"'0.0", regex=True)
df['Контент_рейтинг'] = df['Контент_рейтинг'].replace("'","", regex=True)
df['Контент_рейтинг'] = df['Контент_рейтинг'].astype(float)
df['Артикул'] = df['Артикул'].replace("'","", regex=True).astype(str)
df['Ozon_Product_ID'] = df['Ozon_Product_ID'].astype(str)
df['FBO_OZON_SKU_ID'] = df['FBO_OZON_SKU_ID'].astype(str)
df['FBS_OZON_SKU_ID'] = df['FBS_OZON_SKU_ID'].astype(str)
df.rename(columns={"Ozon_Product_ID": "Ozon_ID",
                   "FBO_OZON_SKU_ID":"FBO_ID",
                   "FBS_OZON_SKU_ID":"FBS_ID"}, inplace=True)
float2int(df)

In [17]:
df.dtypes

Артикул            object
Ozon_ID            object
FBO_ID             object
FBS_ID             object
Контент_рейтинг     int64
Текущая_цена        int64
dtype: object

In [18]:
df.head(3)

,Артикул,Ozon_ID,FBO_ID,FBS_ID,Контент_рейтинг,Текущая_цена
0,30273,105606938,298131117,298131122,15,1200
1,30383,105742204,298264198,298264234,38,1990
2,31284,112525698,303785665,303785701,30,7900


In [19]:
all_df['База'] = df

###### Графики

In [20]:
xls = '/content/base_for_analiz/Ozon/Аналитика Графики 01.06.2022-31.05.2023.xlsx'
graf = pd.read_excel(xls)

In [78]:
df = graf.copy()

In [79]:
normal_col(df)


['Категория_3го_уровня', 'Товары', 'Модель', 'Ozon_ID', 'Артикул', 'День', 'Уникальные_посетители_всего', 'Уникальные_посетители_с_просмотром_карточки_товара', 'Показы_всего', 'Показы_на_карточке_товара', 'Показы_в_поиске_и_каталоге', 'Позиция_в_поиске_и_каталоге', 'В_корзину_всего', 'Заказано_товаров', 'Отменено_товаров', 'Доставлено_за_выбранный_период', 'Возвращено_товаров', 'Заказано_на_сумму', 'Отправления_с_премиумподпиской_принятые_в_обработку']


<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)


In [80]:
def get_index(df):
  df['s_index'] = df['Ozon_ID'].astype(str) + df['День'].astype(str)

In [81]:
get_index(df)

In [82]:
df['Ozon_ID'] = df['Ozon_ID'].astype(str)
# graf['s_index'] = (graf['Ozon_ID']+graf['День']).replace('-','', regex=True).astype(str)
# Сборный индекс - для удобной группировки
#graf['s_index'] = ''.join(f'{graf['Ozon_ID']}_{graf['День']}').replace('-','', regex=True).astype(str)

df['День'] = pd.to_datetime(df['День'])
df['Позиция_в_поиске_и_каталоге'] = df['Позиция_в_поиске_и_каталоге'].replace(',','.', regex=True)
df['Позиция_в_поиске_и_каталоге'] = df['Позиция_в_поиске_и_каталоге'].astype(float)


In [83]:
float2int(df)

In [85]:
df.columns

Index(['Категория_3го_уровня', 'Товары', 'Модель', 'Ozon_ID', 'Артикул',
       'День', 'Уникальные_посетители_всего',
       'Уникальные_посетители_с_просмотром_карточки_товара', 'Показы_всего',
       'Показы_на_карточке_товара', 'Показы_в_поиске_и_каталоге',
       'Позиция_в_поиске_и_каталоге', 'В_корзину_всего', 'Заказано_товаров',
       'Отменено_товаров', 'Доставлено_за_выбранный_период',
       'Возвращено_товаров', 'Заказано_на_сумму',
       'Отправления_с_премиумподпиской_принятые_в_обработку', 's_index'],
      dtype='object')

In [86]:
df.drop(['Товары', 'Модель',
       'Показы_на_карточке_товара', 'Показы_в_поиске_и_каталоге',
       'Отправления_с_премиумподпиской_принятые_в_обработку'], axis=1, inplace=True)

In [87]:
df.dtypes

Категория_3го_уровня                                          object
Ozon_ID                                                       object
Артикул                                                       object
День                                                  datetime64[ns]
Уникальные_посетители_всего                                    int64
Уникальные_посетители_с_просмотром_карточки_товара             int64
Показы_всего                                                   int64
Позиция_в_поиске_и_каталоге                                    int64
В_корзину_всего                                                int64
Заказано_товаров                                               int64
Отменено_товаров                                               int64
Доставлено_за_выбранный_период                                 int64
Возвращено_товаров                                             int64
Заказано_на_сумму                                              int64
s_index                           

In [88]:
all_df['Графики'] = df


In [89]:
ddf = all_df['Графики']

In [90]:
ddf.shape

(254748, 15)

In [91]:
len(ddf['s_index'].unique())

254052

In [92]:
ddf.shape[0] - len(ddf['s_index'].unique())

696

In [96]:
ddf.columns

Index(['Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День',
       'Уникальные_посетители_всего',
       'Уникальные_посетители_с_просмотром_карточки_товара', 'Показы_всего',
       'Позиция_в_поиске_и_каталоге', 'В_корзину_всего', 'Заказано_товаров',
       'Отменено_товаров', 'Доставлено_за_выбранный_период',
       'Возвращено_товаров', 'Заказано_на_сумму', 's_index'],
      dtype='object')

Вывод: В базе есть не уникальные сочетания Ozon_ID и День

In [97]:
#ddf.groupby(['День','Ozon_ID']).agg({'Уникальные_посетители_всего':'sum',
t = ddf.groupby(['s_index','Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День']).agg({'Уникальные_посетители_всего':'sum',
                                     'Уникальные_посетители_с_просмотром_карточки_товара':'sum',
                                     'Показы_всего':'sum',
                                     'Позиция_в_поиске_и_каталоге':'min',
                                     'В_корзину_всего':'sum',
                                     'Заказано_товаров':'sum',
                                     'Отменено_товаров':'sum',
                                     'Доставлено_за_выбранный_период':'sum',
                                     'Возвращено_товаров':'sum',
                                     'Заказано_на_сумму':'sum'}).reset_index()

In [98]:
t.columns

Index(['s_index', 'Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День',
       'Уникальные_посетители_всего',
       'Уникальные_посетители_с_просмотром_карточки_товара', 'Показы_всего',
       'Позиция_в_поиске_и_каталоге', 'В_корзину_всего', 'Заказано_товаров',
       'Отменено_товаров', 'Доставлено_за_выбранный_период',
       'Возвращено_товаров', 'Заказано_на_сумму'],
      dtype='object')

In [106]:
re_col = {'Показы_всего': 'Показы',
          'Уникальные_посетители_всего': 'Посетители',
          'Уникальные_посетители_с_просмотром_карточки_товара':'Посетители_на_карточке_товара',
          'В_корзину_всего': 'В_корзину',
          'Заказано_товаров':'Заказано',
          'Заказано_на_сумму':'Оборот_в_месяц_целевой',
          'Доставлено_за_выбранный_период':'Доставлено',
          'Возвращено_товаров':'Возвращено',
          'Отменено_товаров':'Отменено'
          }
t.rename(columns=re_col, inplace=True)

In [ ]:
t

In [108]:
all_df['Графики'] = t

###### Акции  

In [109]:
xls = '/content/base_for_analiz/Ozon/Аналитика Акции 01.06.2022-31.05.2023.xlsx'
act = pd.read_excel(xls)   #, encoding='cp1251' если выдаёт ошибку

In [128]:
df = act.copy()
df.columns

Index(['Категория 3-го уровня', 'Товары', 'Модель', 'Ozon ID', 'Артикул',
       'Акции', 'День', 'Уникальные посетители, всего',
       'Уникальные посетители с просмотром карточки товара', 'Показы, всего',
       'Показы на карточке товара', 'Показы в поиске и каталоге',
       'Позиция в поиске и каталоге', 'В корзину, всего', 'Заказано товаров',
       'Доставлено (за выбранный период)', 'Заказано на сумму',
       'Заказано товаров по акции', 'Заказано на сумму по акции',
       'Отправления с премиум-подпиской, принятые в обработку'],
      dtype='object')

In [129]:
col = normal_col(df)
df.columns

['Категория_3го_уровня', 'Товары', 'Модель', 'Ozon_ID', 'Артикул', 'Акции', 'День', 'Уникальные_посетители_всего', 'Уникальные_посетители_с_просмотром_карточки_товара', 'Показы_всего', 'Показы_на_карточке_товара', 'Показы_в_поиске_и_каталоге', 'Позиция_в_поиске_и_каталоге', 'В_корзину_всего', 'Заказано_товаров', 'Доставлено_за_выбранный_период', 'Заказано_на_сумму', 'Заказано_товаров_по_акции', 'Заказано_на_сумму_по_акции', 'Отправления_с_премиумподпиской_принятые_в_обработку']


<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)


Index(['Категория_3го_уровня', 'Товары', 'Модель', 'Ozon_ID', 'Артикул',
       'Акции', 'День', 'Уникальные_посетители_всего',
       'Уникальные_посетители_с_просмотром_карточки_товара', 'Показы_всего',
       'Показы_на_карточке_товара', 'Показы_в_поиске_и_каталоге',
       'Позиция_в_поиске_и_каталоге', 'В_корзину_всего', 'Заказано_товаров',
       'Доставлено_за_выбранный_период', 'Заказано_на_сумму',
       'Заказано_товаров_по_акции', 'Заказано_на_сумму_по_акции',
       'Отправления_с_премиумподпиской_принятые_в_обработку'],
      dtype='object')

In [130]:
cols = ['Категория_3го_уровня','Товары', 'Модель', 'Артикул', 'Уникальные_посетители_всего',
       'Уникальные_посетители_с_просмотром_карточки_товара', 'Показы_всего',
       'Показы_на_карточке_товара', 'Показы_в_поиске_и_каталоге',
       'Позиция_в_поиске_и_каталоге', 'В_корзину_всего','Заказано_товаров_по_акции',
       'Доставлено_за_выбранный_период','Заказано_на_сумму_по_акции',
       'Отправления_с_премиумподпиской_принятые_в_обработку']
df.drop(cols, axis=1, inplace=True)
df.head(3)

,Ozon_ID,Акции,День,Заказано_товаров,Заказано_на_сумму
0,882353427,NaN,2023-03-05,0,0
1,882353427,NaN,2023-03-06,0,0
2,882353427,NaN,2023-03-07,0,0


In [134]:
re_col = {'Заказано_товаров': 'Заказано_по_Акции','Заказано_на_сумму':'Заказано_на_сумму_по_акции'}
df.rename(columns=re_col, inplace=True)

In [135]:
# Сборный индекс - для удобной группировки
get_index(df)
float2int(df)

In [136]:
df['Ozon_ID'] = df['Ozon_ID'].astype(str)
# act['Артикул'] = act['Артикул'].astype(str)

df['День'] = pd.to_datetime(df['День'])
df['Акции'] = np.where(df.notna()['Акции'],1,0)

In [139]:
df.dtypes

Ozon_ID                               object
Акции                                  int64
День                          datetime64[ns]
Заказано_по_Акции                      int64
Заказано_на_сумму_по_акции             int64
s_index                               object
dtype: object

In [137]:
df.columns

Index(['Ozon_ID', 'Акции', 'День', 'Заказано_по_Акции',
       'Заказано_на_сумму_по_акции', 's_index'],
      dtype='object')

In [141]:
all_df['Акции'] = df.copy() # до группировки

In [143]:
#t = df.groupby(['s_index', 'День','Ozon_ID', 'Акции']).agg({'Заказано_товаров':'sum',
t = df.groupby('s_index').agg({'Акции':'max','Заказано_по_Акции':'sum',
                                'Заказано_на_сумму_по_акции':'sum'}).reset_index()

In [ ]:
t

In [147]:
t[t['Заказано_по_Акции']>0]

,s_index,Акции,Заказано_по_Акции,Заказано_на_сумму_по_акции
16,2609120352022-11-02,1,2,6080
74,2609196922022-08-21,1,2,9880
77,2609196922022-08-24,1,2,9880
107,2609196922022-09-27,1,2,9880
115,2609196922022-10-05,1,1,4940
...,...,...,...,...
239098,9315290582023-05-16,1,1,1640
239108,9315290582023-05-26,1,1,1640
239155,9323847992023-05-22,1,1,4900
239273,9660180192023-05-10,1,1,1640


In [149]:
all_df['Акции_груп'] = t.copy() # с группировкой

###### Объединяем основные таблицы

In [150]:
df_svod = pd.DataFrame()

In [151]:
print(len(all_df['Графики']))
all_df['Графики'].columns

254069


Index(['s_index', 'Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День',
       'Посетители', 'Посетители_на_карточке_товара', 'Показы',
       'Позиция_в_поиске_и_каталоге', 'В_корзину', 'Заказано', 'Отменено',
       'Доставлено', 'Возвращено', 'Оборот_в_месяц_целевой'],
      dtype='object')

In [152]:
print(len(all_df['Акции_груп']))
all_df['Акции_груп'].columns

239295


Index(['s_index', 'Акции', 'Заказано_по_Акции', 'Заказано_на_сумму_по_акции'], dtype='object')

In [155]:
#df_svod = pd.merge(all_df['Графики'],all_df['Акции'], on=['День', 'Ozon_ID'], how='left')
df_svod = pd.merge(all_df['Графики'],all_df['Акции_груп'], on='s_index', how='left')

In [156]:
df_svod.columns

Index(['s_index', 'Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День',
       'Посетители', 'Посетители_на_карточке_товара', 'Показы',
       'Позиция_в_поиске_и_каталоге', 'В_корзину', 'Заказано', 'Отменено',
       'Доставлено', 'Возвращено', 'Оборот_в_месяц_целевой', 'Акции',
       'Заказано_по_Акции', 'Заказано_на_сумму_по_акции'],
      dtype='object')

In [157]:
df = df_svod.copy()

In [158]:
df.columns

Index(['s_index', 'Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День',
       'Посетители', 'Посетители_на_карточке_товара', 'Показы',
       'Позиция_в_поиске_и_каталоге', 'В_корзину', 'Заказано', 'Отменено',
       'Доставлено', 'Возвращено', 'Оборот_в_месяц_целевой', 'Акции',
       'Заказано_по_Акции', 'Заказано_на_сумму_по_акции'],
      dtype='object')

Есть вопросы по колонкам из Акций:

'Заказано_по_акции',

'Заказано_на_сумму_y',

'Заказано_товаров_по_акции',

'Заказано_на_сумму_по_акции'

In [159]:
df['Средний_чек'] = df['Оборот_в_месяц_целевой'] / df['Заказано']

In [160]:
df.dtypes

s_index                                  object
Категория_3го_уровня                     object
Ozon_ID                                  object
Артикул                                  object
День                             datetime64[ns]
Посетители                                int64
Посетители_на_карточке_товара             int64
Показы                                    int64
Позиция_в_поиске_и_каталоге               int64
В_корзину                                 int64
Заказано                                  int64
Отменено                                  int64
Доставлено                                int64
Возвращено                                int64
Оборот_в_месяц_целевой                    int64
Акции                                   float64
Заказано_по_Акции                       float64
Заказано_на_сумму_по_акции              float64
Средний_чек                             float64
dtype: object

In [161]:
float2int(df)

In [ ]:
df.isna().mean().sort_values(ascending=False)

In [165]:
df.head(3)

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,Заказано,Отменено,Доставлено,Возвращено,Оборот_в_месяц_целевой,Акции,Заказано_по_Акции,Заказано_на_сумму_по_акции,Средний_чек
0,2609022712022-06-02,Лампа автомобильная,260902271,25541,2022-06-02,1,0,1,0,0,0,0,0,0,0,0,0,0,0
1,2609022712022-06-03,Лампа автомобильная,260902271,25541,2022-06-03,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2609022712022-06-07,Лампа автомобильная,260902271,25541,2022-06-07,1,1,1,0,0,0,0,0,0,0,0,0,0,0


In [164]:
df.columns

Index(['s_index', 'Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День',
       'Посетители', 'Посетители_на_карточке_товара', 'Показы',
       'Позиция_в_поиске_и_каталоге', 'В_корзину', 'Заказано', 'Отменено',
       'Доставлено', 'Возвращено', 'Оборот_в_месяц_целевой', 'Акции',
       'Заказано_по_Акции', 'Заказано_на_сумму_по_акции', 'Средний_чек'],
      dtype='object')

In [166]:
df.shape

(254069, 19)

In [ ]:
df.groupby(['s_index', 'Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День']).agg({'Посетители':'sum',
                                    'Посетители_на_карточке_товара':'sum',
                                    'Показы':'sum',
                                    'Позиция_в_поиске_и_каталоге':'min',
                                    'В_корзину':'sum', 'Заказано':'sum',
                                    'Отменено':'sum',
                                    'Доставлено':'sum',
                                    'Возвращено':'sum',
                                    'Оборот_в_месяц_целевой':'sum',
                                    'Акции':'max',
                                    'Заказано_по_Акции':'sum',
                                    'Заказано_на_сумму_по_акции':'sum',
                                    'Средний_чек':'mean'}).reset_index()

In [169]:
df.shape

(254069, 19)

In [170]:
mpoz = df['Позиция_в_поиске_и_каталоге'].max()
mpoz

4584

In [171]:
df['Позиция_в_поиске_и_каталоге'] = np.where((df['Позиция_в_поиске_и_каталоге']==0),
                                             mpoz, df['Позиция_в_поиске_и_каталоге'])

In [172]:
df[df['Позиция_в_поиске_и_каталоге'] == 0] # Проверяем

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,Заказано,Отменено,Доставлено,Возвращено,Оборот_в_месяц_целевой,Акции,Заказано_по_Акции,Заказано_на_сумму_по_акции,Средний_чек


In [173]:
df.columns

Index(['s_index', 'Категория_3го_уровня', 'Ozon_ID', 'Артикул', 'День',
       'Посетители', 'Посетители_на_карточке_товара', 'Показы',
       'Позиция_в_поиске_и_каталоге', 'В_корзину', 'Заказано', 'Отменено',
       'Доставлено', 'Возвращено', 'Оборот_в_месяц_целевой', 'Акции',
       'Заказано_по_Акции', 'Заказано_на_сумму_по_акции', 'Средний_чек'],
      dtype='object')

In [174]:
baza = all_df['База']
baza.head(3)

,Артикул,Ozon_ID,FBO_ID,FBS_ID,Контент_рейтинг,Текущая_цена
0,30273,105606938,298131117,298131122,15,1200
1,30383,105742204,298264198,298264234,38,1990
2,31284,112525698,303785665,303785701,30,7900


In [175]:
len(baza['Артикул'].unique())

1259

In [176]:
len(baza['Ozon_ID'].unique())

1259

In [177]:
len(df['Ozon_ID'].unique())

1233

In [178]:
len(df['Артикул'].unique())

994

In [179]:
len(baza['FBS_ID'].unique())

1249

In [180]:
baza.dtypes

Артикул            object
Ozon_ID            object
FBO_ID             object
FBS_ID             object
Контент_рейтинг     int64
Текущая_цена        int64
dtype: object

In [181]:
df.dtypes

s_index                                  object
Категория_3го_уровня                     object
Ozon_ID                                  object
Артикул                                  object
День                             datetime64[ns]
Посетители                                int64
Посетители_на_карточке_товара             int64
Показы                                    int64
Позиция_в_поиске_и_каталоге               int64
В_корзину                                 int64
Заказано                                  int64
Отменено                                  int64
Доставлено                                int64
Возвращено                                int64
Оборот_в_месяц_целевой                    int64
Акции                                     int64
Заказано_по_Акции                         int64
Заказано_на_сумму_по_акции                int64
Средний_чек                               int64
dtype: object

In [182]:
uniq_b = (baza['FBS_ID'].unique())
uniq_s = (df['Ozon_ID'].unique())
w = []
for i in uniq_b:
  for j in uniq_s:
    if i == j:
      w.append(j)

print(f'Уникальных в Базе: {len(uniq_b)}')
print(f'Уникальных в Своде: {len(uniq_s)}')
print(f'Общих: {len(w)}')

Уникальных в Базе: 1249
Уникальных в Своде: 1233
Общих: 923


In [183]:
b = baza[['FBS_ID','Контент_рейтинг','Текущая_цена']].copy()

In [184]:
b.rename(columns={'FBS_ID':'Ozon_ID'}, inplace=True)

In [186]:
b.head(3)

,Ozon_ID,Контент_рейтинг,Текущая_цена
0,298131122,15,1200
1,298264234,38,1990
2,303785701,30,7900


In [187]:
df.head(3)

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,Заказано,Отменено,Доставлено,Возвращено,Оборот_в_месяц_целевой,Акции,Заказано_по_Акции,Заказано_на_сумму_по_акции,Средний_чек
0,2609022712022-06-02,Лампа автомобильная,260902271,25541,2022-06-02,1,0,1,4584,0,0,0,0,0,0,0,0,0,0
1,2609022712022-06-03,Лампа автомобильная,260902271,25541,2022-06-03,1,0,1,4584,0,0,0,0,0,0,0,0,0,0
2,2609022712022-06-07,Лампа автомобильная,260902271,25541,2022-06-07,1,1,1,4584,0,0,0,0,0,0,0,0,0,0


In [188]:
df_svod = pd.merge(df,b, on='Ozon_ID', how='left')

In [190]:
float2int(df_svod)
df_svod.head(3)

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,...,Отменено,Доставлено,Возвращено,Оборот_в_месяц_целевой,Акции,Заказано_по_Акции,Заказано_на_сумму_по_акции,Средний_чек,Контент_рейтинг,Текущая_цена
0,2609022712022-06-02,Лампа автомобильная,260902271,25541,2022-06-02,1,0,1,4584,0,...,0,0,0,0,0,0,0,0,0,0
1,2609022712022-06-03,Лампа автомобильная,260902271,25541,2022-06-03,1,0,1,4584,0,...,0,0,0,0,0,0,0,0,0,0
2,2609022712022-06-07,Лампа автомобильная,260902271,25541,2022-06-07,1,1,1,4584,0,...,0,0,0,0,0,0,0,0,0,0


In [191]:
df_svod.to_csv('/content/df_svod.csv')

###### Ozon/Performance/Трафареты

In [192]:
all_traf = pd.DataFrame()

In [193]:
for i in traf_paths:
  df = pd.read_csv(traf_paths[i], skiprows=[0], delimiter=';') #, skiprows=[0,2], usecols=['column1', 'column2']
  print('*'*40)
  print(f'df - {i}')
  print(df.shape)
  print(df.columns)
  normal_col(df)
  df.dropna(subset=['sku'], inplace=True) # удаляем строки с пустым ID
  try:
    cols = ['Название_товара', 'Цена_товара',
          'CTR_%', 'Ср_цена_1000_показов']
    df.drop(cols, axis=1, inplace=True)
  except:
    cols = ['Название_товара', 'Цена_товара','CTR_%','Ср_цена_клика']
    df.drop(cols, axis=1, inplace=True)

  df['sku'] = df['sku'].astype(int)
  df.rename(columns={'sku': 'Ozon_ID'}, inplace=True)
  df['День'] = pd.to_datetime(df['День'])
  df['Ozon_ID'] = df['Ozon_ID'].astype(str)
  df['Расход__с_НДС'] = df['Расход__с_НДС'].replace(',','.', regex=True)
  df['Расход__с_НДС'] = df['Расход__с_НДС'].astype(float)
  df['Выручка'] = df['Выручка'].replace(',','.', regex=True)
  df['Выручка'] = df['Выручка'].astype(float)
  df['Выручка_с_заказов_модели'] = df['Выручка_с_заказов_модели'].replace(',','.', regex=True)
  df['Выручка_с_заказов_модели'] = df['Выручка_с_заказов_модели'].astype(float)
  if df.shape[0] != 1:
    all_traf = pd.concat([all_traf, df], ignore_index=True)

****************************************
df - 3264126
(276, 13)
Index(['День', 'sku', 'Название товара', 'Цена товара, ₽', 'Показы', 'Клики',
       'CTR (%)', 'Ср. цена 1000 показов, ₽', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Название_товара', 'Цена_товара', 'Показы', 'Клики', 'CTR_%', 'Ср_цена_1000_показов', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_модели', 'Выручка_с_заказов_модели']
****************************************
df - 3263861
(255, 13)
Index(['День', 'sku', 'Название товара', 'Цена товара, ₽', 'Показы', 'Клики',
       'CTR (%)', 'Ср. цена 1000 показов, ₽', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Название_товара', 'Цена_товара', 'Показы', 'Клики', 'CTR_%', 'Ср_цена_1000_показов', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_модели', 'Выручка_с_заказов_модели']
**********

<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-193-5f4a22510bdf>:19: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['День'] = pd.to_datetime(df['День'])
<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-193-5f4a22510bdf>:19: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['День'] = pd.to_datetime(df['Де

****************************************
df - 1616775_23.09.2022-23.11.2022
(333, 13)
Index(['День', 'sku', 'Название товара', 'Цена товара, ₽', 'Показы', 'Клики',
       'CTR (%)', 'Ср. цена 1000 показов, ₽', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Название_товара', 'Цена_товара', 'Показы', 'Клики', 'CTR_%', 'Ср_цена_1000_показов', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_модели', 'Выручка_с_заказов_модели']
****************************************
df - 1616775_26.03.2023-26.05.2023
(1, 13)
Index(['День', 'sku', 'Название товара', 'Цена товара, ₽', 'Показы', 'Клики',
       'CTR (%)', 'Ср. цена 1000 показов, ₽', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Название_товара', 'Цена_товара', 'Показы', 'Клики', 'CTR_%', 'Ср_цена_1000_показов', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_модел

<ipython-input-193-5f4a22510bdf>:19: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['День'] = pd.to_datetime(df['День'])
<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-193-5f4a22510b

****************************************
df - 1223100_27.04.2023-31.05.2023
(1, 13)
Index(['День', 'sku', 'Название товара', 'Цена товара, ₽', 'Показы', 'Клики',
       'CTR (%)', 'Ср. цена 1000 показов, ₽', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Название_товара', 'Цена_товара', 'Показы', 'Клики', 'CTR_%', 'Ср_цена_1000_показов', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_модели', 'Выручка_с_заказов_модели']
****************************************
df - 1223100_23.08.2022-23.10.2022
(12015, 14)
Index(['День', 'sku', 'Название товара', 'Цена товара, ₽', 'Показы', 'Клики',
       'CTR (%)', 'Ср. цена 1000 показов, ₽', 'Расход, ₽, с НДС',
       'Расход за минусом бонусов, ₽, с НДС', 'Заказы', 'Выручка, ₽',
       'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Название_товара', 'Цена_товара', 'Показы', 'Клики', 'CTR_%', 'Ср_цена_1000_показов', 'Ра

<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-193-5f4a22510bdf>:19: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['День'] = pd.to_datetime(df['День'])
<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-7-06ccb71f2ffc

****************************************
df - 1615846_23.09.2022-23.11.2022
(26385, 13)
Index(['День', 'sku', 'Название товара', 'Цена товара, ₽', 'Показы', 'Клики',
       'CTR (%)', 'Ср. цена 1000 показов, ₽', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Название_товара', 'Цена_товара', 'Показы', 'Клики', 'CTR_%', 'Ср_цена_1000_показов', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_модели', 'Выручка_с_заказов_модели']
****************************************
df - 1615846_24.11.2022-24.01.2023
(1426, 13)
Index(['День', 'sku', 'Название товара', 'Цена товара, ₽', 'Показы', 'Клики',
       'CTR (%)', 'Ср. цена 1000 показов, ₽', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Название_товара', 'Цена_товара', 'Показы', 'Клики', 'CTR_%', 'Ср_цена_1000_показов', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_

<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-193-5f4a22510bdf>:19: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['День'] = pd.to_datetime(df['День'])
<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-193-5f4a22510bdf>:19: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['День'] = pd.to_datetime(df['Де

****************************************
df - 1615846_25.01.2023-25.03.2023
(1027, 13)
Index(['День', 'sku', 'Название товара', 'Цена товара, ₽', 'Показы', 'Клики',
       'CTR (%)', 'Ср. цена 1000 показов, ₽', 'Расход, ₽, с НДС', 'Заказы',
       'Выручка, ₽', 'Заказы модели', 'Выручка с заказов модели, ₽'],
      dtype='object')
['День', 'sku', 'Название_товара', 'Цена_товара', 'Показы', 'Клики', 'CTR_%', 'Ср_цена_1000_показов', 'Расход__с_НДС', 'Заказы', 'Выручка', 'Заказы_модели', 'Выручка_с_заказов_модели']


<ipython-input-7-06ccb71f2ffc>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(cols, axis=1, inplace=True)
<ipython-input-193-5f4a22510bdf>:19: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['День'] = pd.to_datetime(df['День'])


In [194]:
all_traf.drop(['Расход_за_минусом_бонусов__с_НДС'], axis=1, inplace=True)

In [195]:
all_traf['Трафареты_Количество_заказов'] = all_traf['Заказы']+all_traf['Заказы_модели']
all_traf['Трафареты_Сумма_заказов'] = all_traf['Выручка']+all_traf['Выручка_с_заказов_модели']

In [196]:
col = {'Показы':'Трафареты_Охват',
       'Клики':'Трафареты_Клики',
       'Расход__с_НДС':'Трафареты_Расходы_всего'}
all_traf.rename(columns=col, inplace=True)
all_traf.drop(['Заказы','Выручка', 'Заказы_модели', 'Выручка_с_заказов_модели'], axis=1, inplace=True)

In [197]:
all_traf.columns

Index(['День', 'Ozon_ID', 'Трафареты_Охват', 'Трафареты_Клики',
       'Трафареты_Расходы_всего', 'Трафареты_Количество_заказов',
       'Трафареты_Сумма_заказов'],
      dtype='object')

In [198]:
df = all_traf.copy()

In [199]:
df.dtypes

День                            datetime64[ns]
Ozon_ID                                 object
Трафареты_Охват                        float64
Трафареты_Клики                        float64
Трафареты_Расходы_всего                float64
Трафареты_Количество_заказов           float64
Трафареты_Сумма_заказов                float64
dtype: object

In [200]:
float2int(df)

In [201]:
get_index(df)

In [202]:
df.drop(['День','Ozon_ID'], axis=1, inplace=True)

In [203]:
df

,Трафареты_Охват,Трафареты_Клики,Трафареты_Расходы_всего,Трафареты_Количество_заказов,Трафареты_Сумма_заказов,s_index
0,219,9,2,0,0,5115723402023-03-05
1,71,1,1,0,0,5115723642023-03-05
2,432,5,4,0,0,5115725212023-03-05
3,74,1,1,0,0,5115728192023-03-05
4,1078,22,11,0,0,5115730082023-03-05
...,...,...,...,...,...,...
56242,204,2,31,0,0,6647634252023-07-03
56243,188,1,31,0,0,6647732272023-07-03
56244,282,6,54,0,0,7264762712023-07-03
56245,221,9,38,0,0,7638314802023-07-03


In [204]:
all_df['Трафареты'] = df.copy()

Добавляем к Сводному ДФ

In [205]:
df_svod = pd.merge(df_svod,df, on='s_index', how='left')

In [206]:
df_svod

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,...,Заказано_по_Акции,Заказано_на_сумму_по_акции,Средний_чек,Контент_рейтинг,Текущая_цена,Трафареты_Охват,Трафареты_Клики,Трафареты_Расходы_всего,Трафареты_Количество_заказов,Трафареты_Сумма_заказов
0,2609022712022-06-02,Лампа автомобильная,260902271,25541,2022-06-02,1,0,1,4584,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,2609022712022-06-03,Лампа автомобильная,260902271,25541,2022-06-03,1,0,1,4584,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,2609022712022-06-07,Лампа автомобильная,260902271,25541,2022-06-07,1,1,1,4584,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
3,2609022712022-06-08,Лампа автомобильная,260902271,25541,2022-06-08,0,0,0,4584,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,2609022712022-06-12,Лампа автомобильная,260902271,25541,2022-06-12,1,0,1,4584,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254466,9660743052023-05-28,Лампа автомобильная,966074305,34599,2023-05-28,19,0,19,132,0,...,0,0,0,40,1640,NaN,NaN,NaN,NaN,NaN
254467,9660743052023-05-29,Лампа автомобильная,966074305,34599,2023-05-29,26,1,28,155,0,...,0,0,0,40,1640,NaN,NaN,NaN,NaN,NaN
254468,9660743052023-05-30,Лампа автомобильная,966074305,34599,2023-05-30,36,3,43,59,0,...,0,0,0,40,1640,NaN,NaN,NaN,NaN,NaN
254469,9660743052023-05-31,Лампа автомобильная,966074305,34599,2023-05-31,21,3,30,144,1,...,0,0,0,40,1640,NaN,NaN,NaN,NaN,NaN


###### Ozon/Performance/ПВП

In [207]:
all_pvp = pd.DataFrame()

In [ ]:
for i in pvp_paths:
  df = pd.read_csv(pvp_paths[i], skiprows=[0], delimiter=';') #, skiprows=[0,2], usecols=['column1', 'column2']
  print('*'*40)
  print(f'df - {i}')
  print(df.shape)
  print(df.columns)
  normal_col(df)
  df.dropna(subset=['Ozon_ID'], inplace=True) # удаляем строки с пустым ID
  try:
    cols = ['Номер_заказа', 'ID_заказа','Ozon_ID_рекламируемого_товара',
            'Наименование','Расход']
    df.drop(cols, axis=1, inplace=True)
  except:
    # cols = ['Название_товара', 'Цена_товара','CTR','Ср_цена_клика']
    # df.drop(cols, axis=1, inplace=True)
    print('Шо-то тут не то')
  df['Ozon_ID'] = df['Ozon_ID'].astype(int)
  df['Дата'] = pd.to_datetime(df['Дата'])
  df.rename(columns={'Дата': 'День'}, inplace=True)
  df['Ozon_ID'] = df['Ozon_ID'].astype(str)
  df['Ставка'] = df['Ставка'].replace(',','.', regex=True)
  df['Ставка'] = df['Ставка'].astype(float)
  df['Ставка_%'] = df['Ставка_%'].replace(',','.', regex=True)
  df['Ставка_%'] = df['Ставка_%'].astype(float)
  df['Стоимость'] = df['Стоимость'].replace(',','.', regex=True)
  df['Стоимость'] = df['Стоимость'].astype(float)
  df['Цена_продажи'] = df['Цена_продажи'].replace(',','.', regex=True)
  df['Цена_продажи'] = df['Цена_продажи'].astype(float)
  df['Количество'] = df['Количество'].astype(int)
  if df.shape[0] > 1:
    all_pvp = pd.concat([all_pvp, df], ignore_index=True)
  print(df.dtypes)

In [211]:
df = all_pvp.copy()
df.shape

(3676, 8)

In [210]:
df.head(3)

,День,Ozon_ID,Артикул,Количество,Цена_продажи,Стоимость,Ставка_%,Ставка
0,2023-04-21,930789187,34536.0,1,1590.0,1590.0,8.0,127.20
1,2023-05-16,931529058,34538.0,1,1640.0,1640.0,8.0,131.20
2,2022-11-23,275493676,29276.0,1,7751.0,7751.0,5.0,387.55


In [227]:
col = {'Количество':'Продвижение_в_поиске_Количество_заказов',
       'Стоимость':'Продвижение_в_поиске_Сумма_заказов',
       'Ставка':'Продвижение_в_поиске_Ставка'}
df.rename(columns=col, inplace=True)

In [ ]:
get_index(df)

In [228]:
df.columns

Index(['Продвижение_в_поиске_Количество_заказов', 'Цена_продажи',
       'Продвижение_в_поиске_Сумма_заказов', 'Ставка_%',
       'Продвижение_в_поиске_Ставка', 's_index'],
      dtype='object')

In [217]:
uniq_b = (df['Ozon_ID'].unique())
uniq_s = (df_svod['Ozon_ID'].unique())
w = []
for i in uniq_b:
  for j in uniq_s:
    if i == j:
      w.append(j)

print(f'Уникальных в БазеПВП: {len(uniq_b)}')
print(f'Уникальных в Своде: {len(uniq_s)}')
print(f'Общих: {len(w)}')

Уникальных в БазеПВП: 429
Уникальных в Своде: 1233
Общих: 429


In [218]:
col = ['День','Ozon_ID', 'Артикул']
df.drop(columns = col, inplace=True)

In [231]:
float2int(df)

In [232]:
df.head(3)

,Продвижение_в_поиске_Количество_заказов,Цена_продажи,Продвижение_в_поиске_Сумма_заказов,Ставка_%,Продвижение_в_поиске_Ставка,s_index
0,1,1590,1590,8,127,9307891872023-04-21
1,1,1640,1640,8,131,9315290582023-05-16
2,1,7751,7751,5,388,2754936762022-11-23


In [233]:
all_df['ПВП'] = df.copy()

In [234]:
df_svod = pd.merge(df_svod,df, on='s_index', how='left')

In [237]:
float2int(df_svod)

In [238]:
df_svod.head(4)

,s_index,Категория_3го_уровня,Ozon_ID,Артикул,День,Посетители,Посетители_на_карточке_товара,Показы,Позиция_в_поиске_и_каталоге,В_корзину,...,Трафареты_Охват,Трафареты_Клики,Трафареты_Расходы_всего,Трафареты_Количество_заказов,Трафареты_Сумма_заказов,Продвижение_в_поиске_Количество_заказов,Цена_продажи,Продвижение_в_поиске_Сумма_заказов,Ставка_%,Продвижение_в_поиске_Ставка
0,2609022712022-06-02,Лампа автомобильная,260902271,25541,2022-06-02,1,0,1,4584,0,...,0,0,0,0,0,0,0,0,0,0
1,2609022712022-06-03,Лампа автомобильная,260902271,25541,2022-06-03,1,0,1,4584,0,...,0,0,0,0,0,0,0,0,0,0
2,2609022712022-06-07,Лампа автомобильная,260902271,25541,2022-06-07,1,1,1,4584,0,...,0,0,0,0,0,0,0,0,0,0
3,2609022712022-06-08,Лампа автомобильная,260902271,25541,2022-06-08,0,0,0,4584,0,...,0,0,0,0,0,0,0,0,0,0


###### 'Ozon/Товары по категориям'

In [ ]:
all_cat = pd.DataFrame()

In [ ]:
cat_paths['2023-06-05 Автомобильные лампы ксенон комплектующие']

'/content/base_for_analiz/Ozon/Товары по категориям/2023-06-05 Автомобильные лампы ксенон комплектующие.xlsx'

In [ ]:
for i in cat_paths:
  df = pd.read_csv(cat_paths[i], skiprows=[0], delimiter=';') #, skiprows=[0,2], usecols=['column1', 'column2']
  print('*'*40)
  print(f'df - {i}')
  print(df.shape)
  print(df.columns)
  normal_col(df)
  df.dropna(subset=['Ozon_ID'], inplace=True) # удаляем строки с пустым ID
  try:
    cols = ['Номер_заказа', 'ID_заказа','Ozon_ID_рекламируемого_товара',
            'Наименование','Расход']
    df.drop(cols, axis=1, inplace=True)
  except:
    # cols = ['Название_товара', 'Цена_товара','CTR','Ср_цена_клика']
    # df.drop(cols, axis=1, inplace=True)
    print('Шо-то тут не то')
  df['Ozon_ID'] = df['Ozon_ID'].astype(int)
  df['Дата'] = pd.to_datetime(df['Дата'])
  df.rename(columns={'Дата': 'День'}, inplace=True)
  df['Ozon_ID'] = df['Ozon_ID'].astype(str)
  df['Ставка'] = df['Ставка'].replace(',','.', regex=True)
  df['Ставка'] = df['Ставка'].astype(float)
  df['Ставка_%'] = df['Ставка_%'].replace(',','.', regex=True)
  df['Ставка_%'] = df['Ставка_%'].astype(float)
  df['Стоимость'] = df['Стоимость'].replace(',','.', regex=True)
  df['Стоимость'] = df['Стоимость'].astype(float)
  df['Цена_продажи'] = df['Цена_продажи'].replace(',','.', regex=True)
  df['Цена_продажи'] = df['Цена_продажи'].astype(float)
  df['Количество'] = df['Количество'].astype(int)
  if df.shape[0] > 1:
    all_pvp = pd.concat([all_pvp, df], ignore_index=True)
  print(df.dtypes)

In [ ]:
df = pd.read_excel(cat_paths['2023-06-05 Автомобильные лампы ксенон комплектующие'], skiprows=[0,2]) #, skiprows=[0,2], usecols=['column1', 'column2']

In [ ]:
df.shape

(22, 3)

In [ ]:
df.dtypes